In [3]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from torch.utils.data import Dataset, random_split
import torchvision.io as io
import torch
import torchvision
from torch import nn

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

Mounted at /content/drive


In [10]:
def accuracy(prediction, label):
    """
    Compute the accuracy of the prediction
    
    @param prediction : the prediction of the model, int64 tensor of shape (batch_size), either 0 or 1
    @param label      : the labels of the data     , int64 tensor of shape (batch_size), either 0 or 1
    """
    
    batch_size = label.size(0)
    correct = torch.sum(prediction == label)
    return (correct / batch_size).cpu()

def F1_score(prediction, label):
    """
    Compute the F1-score of the prediction
    
    @param prediction : the prediction of the model, int64 tensor of shape (batch_size), either 0 or 1
    @param label      : the labels of the data     , int64 tensor of shape (batch_size), either 0 or 1
    """
    
    batch_size = label.size(0)
    
    precision = (torch.sum(prediction * label) / torch.sum(prediction))
    recall = (torch.sum(prediction * label) / torch.sum(label))
    
    F1 = 2 * precision * recall / (precision + recall)
    return F1.cpu().item()


In [34]:
def train(model, criterion, dataset_train, dataset_test, optimizer, num_epochs):
    """
    Train the given model
    
    @param model         : torch.nn.Module
    @param criterion     : torch.nn.modules.loss._Loss
    @param dataset_train : torch.utils.data.DataLoader
    @param dataset_test  : torch.utils.data.DataLoader
    @param optimizer     : torch.optim.Optimizer
    @param num_epochs    : int
    """
    print("Starting training")
    model.to(device)
    for epoch in range(num_epochs):
        # Train an epoch
        model.train()
        for batch_x, batch_y in dataset_train:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Evaluate the network (forward pass)
            batch_pred = model(batch_x)
            loss = criterion(batch_pred, batch_y.float())

            # Compute the gradient
            optimizer.zero_grad()
            loss.backward()

            # Update the parameters of the model with a gradient step
            optimizer.step()

        # Test the quality on the test set
        model.eval()
        accuracies_test = []
        f1_scores_test = []
        for batch_x, batch_y in dataset_test:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Evaluate the network (forward pass)
            prediction = model(batch_x)
            accuracies_test.append(accuracy(prediction, batch_y))
            f1_scores_test.append(F1_score(prediction, batch_y))

        print(f"Epoch {epoch + 1 : 2} | Test accuracy : {np.mean(accuracies_test):.5} | Test F1 : {np.mean(f1_scores_test):.5}")

In [39]:
threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

class PatchModel(nn.Module):
    """
    Model that tells if a 16 x 16 RGB (as a 3 x 16 x 16 tensor) correspond to a road (1) or not (0)
    """
    def __init__(self):
        super().__init__()
        
        # 3 channels 16 x 16
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=5
        )
        # 10 channels 12 x 12 (12 = 16 - (kernel_size - 1))
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        # 10 channels 6 x 6 (6 = 12 / kernel_size)
        self.conv2 = nn.Conv2d(
            in_channels=10,
            out_channels=20,
            kernel_size=3
        )
        # 20 channels 4 x 4 (4 = 6 - (kernel_size - 1))
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        # 20 channels 2 x 2 (2 = 4 / kernel_size)
        
        self.lin1 = nn.Linear(
            in_features=20 * 2 * 2,
            out_features=10
        )
        self.lin2 = nn.Linear(
            in_features=10,
            out_features=1
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.pool1(self.conv1(x)))
        x = self.relu(self.pool2(self.conv2(x)))
        x = x.view(-1, 20 * 2 * 2)
        x = self.relu(self.lin1(x))
        x = self.sigmoid(self.lin2(x))
        # If we are in testing mode then the output should be either 0 or 1
        if not self.training:
            x = 1 * (x > treshold)
        return x.view(-1)

In [14]:
!pip install requests

In [15]:
import requests
# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/karimassi/road-segmentation/feature/data/MyDataSet.py?token=AH35XE5O6DUEEWEJA5UTHLK7YUPTG'
r = requests.get(url)

with open('MyDataSet.py', 'w') as f:
    f.write(r.text)

In [28]:
import MyDataSet

In [26]:
torch.tensor(1, dtype=torch.float32)

tensor(1.)

In [41]:
from torchvision import transforms 
num_epochs = 10
learning_rate = 1e-3
batch_size = 100

dataset = MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, treshold)

rotations = [(0, 45), (45, 90), (90, 135), (135, 180), (0, -45), (-45, -90), (-90, -135), (-135, -180)]
for rotation in rotations:
    dataset += MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, treshold, transforms.RandomRotation(rotation))

data_len = len(dataset)
train_len = int(data_len * 0.7)
test_len = int(data_len * 0.3)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])

print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

# Train the logistic regression model with the Adam optimizer
criterion = torch.nn.MSELoss()
model = PatchModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model, criterion, dataloader_train, dataloader_test, optimizer, num_epochs)


43750 18750
Starting training
Epoch  1 | Test accuracy : 0.83213 | Test F1 : nan
Epoch  2 | Test accuracy : 0.83191 | Test F1 : nan
Epoch  3 | Test accuracy : 0.83218 | Test F1 : nan
Epoch  4 | Test accuracy : 0.83638 | Test F1 : nan
Epoch  5 | Test accuracy : 0.87048 | Test F1 : 0.49964
Epoch  6 | Test accuracy : 0.9016 | Test F1 : 0.65668
Epoch  7 | Test accuracy : 0.91 | Test F1 : 0.70151
Epoch  8 | Test accuracy : 0.92527 | Test F1 : 0.75063
Epoch  9 | Test accuracy : 0.92351 | Test F1 : 0.73732
Epoch  10 | Test accuracy : 0.9225 | Test F1 : 0.71207
